In [2]:
!pip install pandas sqlalchemy psycopg2 faker


  Using cached faker-37.3.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 19.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 19.2 MB/s eta 0:00:00
Using cached faker-37.3.0-py3-none-any.whl (1.9 MB)

   ---------- ----------------------------- 1/4 [greenlet]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------------------- 2/4 [faker]
   -------------------- ------

In [1]:
import pandas as pd
import random
import string
from faker import Faker
from sqlalchemy import create_engine

# Initialize Faker
fake = Faker()

# PostgreSQL DB credentials
DB_CONFIG = {
    'user': 'eval_user',
    'password': 'eval_password',
    'host': 'localhost',
    'port': '5432',
    'dbname': 'elevate_db'
}

def generate_unique_code():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=12))

membership_choices = ["S", "P", "UX"]

def generate_membership_data(n=100):
    data = []
    for _ in range(n):
        name = fake.name()
        plan = random.choice(membership_choices)
        active = random.choice([True, False])
        unique_code = generate_unique_code()
        created_at = fake.date_time_this_year()
        updated_at = created_at
        data.append([name, plan, active, created_at, updated_at, unique_code])
    df = pd.DataFrame(data, columns=[
        "name", "membership_plan", "membership_active", "created_at", "updated_at", "unique_code"
    ])
    df.drop_duplicates(subset=['name', 'unique_code'], inplace=True)
    return df

df = generate_membership_data(100)

engine_url = (
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
)
engine = create_engine(engine_url)

df.to_sql("crm_membership", engine, if_exists="append", index=False)
print("✅ Synthetic data inserted successfully!")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "eval_user"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [9]:
pip install python-slugify



   ---------------------------------------- 2/2 [python-slugify]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
from faker import Faker
from slugify import slugify
import random

# Setup Faker
fake = Faker()

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="elevate_db",
    user="eval_user",
    password="eval_password",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Optional: Clear old data
cursor.execute("DELETE FROM education_lesson")
cursor.execute("DELETE FROM education_course")
conn.commit()

# Insert synthetic Course records
course_ids = []
for _ in range(10):
    title = fake.sentence(nb_words=3).rstrip('.')
    description = fake.paragraph(nb_sentences=4)
    start_date = fake.date_this_year()
    end_date = fake.date_between(start_date=start_date, end_date="+60d")
    instructor = fake.name()
    slug = slugify(title)

    cursor.execute('''
        INSERT INTO education_course (
            "Course_title", "Course_description", "Course_start_date",
            "Course_end_date", "Course_instructor", "slug"
        ) VALUES (%s, %s, %s, %s, %s, %s)
        RETURNING id
    ''', (title, description, start_date, end_date, instructor, slug))

    course_id = cursor.fetchone()[0]
    course_ids.append(course_id)

# Insert synthetic Lessons for each Course
for course_id in course_ids:
    for order in range(1, random.randint(3, 6)):
        name = fake.sentence(nb_words=4).rstrip('.')
        content = fake.paragraph(nb_sentences=6)
        slug = slugify(name)

        cursor.execute('''
            INSERT INTO education_lesson (
                lesson_name, lesson_content, lesson_order,
                slug, lesson_course_id
            ) VALUES (%s, %s, %s, %s, %s)
        ''', (name, content, order, slug, course_id))

conn.commit()
cursor.close()
conn.close()

print("✅ Inserted synthetic data into education_course and education_lesson.")


✅ Inserted synthetic data into education_course and education_lesson.
